In [1]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from awsglue.transforms import Map, DropFields, ApplyMapping
import psycopg2

sc = SparkContext()
context = GlueContext(sc)

In [2]:
path = "s3://network.cubo.datalake/airtable/raw/prod/"
path = path + "2025-03-14-21-54-14/data/"
path = path + "Calendário de Eventos Cubo/Events/Plataforma Cubo Network/"
print(path)
dataframeBaseEventos = context.create_dynamic_frame.from_options(
                connection_type='s3',
                connection_options={
                    'paths': [path],
                    'recurse': True
                },
                format='json'
            )
dataframeBaseEventos.count()

s3://network.cubo.datalake/airtable/raw/prod/2025-03-14-21-54-14/data/Calendário de Eventos Cubo/Events/Plataforma Cubo Network/


5

In [3]:
dataframeBaseEventos.printSchema()

root
|-- id: string
|-- createdTime: string
|-- fields: struct
|    |-- Título do evento: string
|    |-- Categoria de evento: string
|    |-- Data e Horário: string
|    |-- Tipo do evento: string
|    |-- Link de inscrição: string
|    |-- Formato: string
|    |-- Trimestre: string
|    |-- Semestre: string
|    |-- Publico: array
|    |    |-- element: string
|    |-- Hub: array
|    |    |-- element: string
|    |-- Imagem: array
|    |    |-- element: struct
|    |    |    |-- id: string
|    |    |    |-- width: int
|    |    |    |-- height: int
|    |    |    |-- url: string
|    |    |    |-- filename: string
|    |    |    |-- size: int
|    |    |    |-- type: string
|    |    |    |-- thumbnails: struct
|    |    |    |    |-- small: struct
|    |    |    |    |    |-- url: string
|    |    |    |    |    |-- width: int
|    |    |    |    |    |-- height: int
|    |    |    |    |-- large: struct
|    |    |    |    |    |-- url: string
|    |    |    |    |    |-- width: 

In [4]:
mapping = []
for item in dataframeBaseEventos.unnest().toDF().dtypes:
    if item[0].split('.')[0] == "fields":
        if len(item[0].split(".")) == 2:
            mapping.append((item[0], item[0].split('.')[1]))
    else:
        mapping.append((item[0], item[0].split('.')[0]))  

Aqui abaixo inicia os testes

In [ ]:
def recInsights(item):
    try:
        # Verifica se "fields" existe no item
        if "fields" in item:
            # Verifica se "Imagem" está presente e contém pelo menos um item no array
            if "Imagem" in item["fields"] and isinstance(item["fields"]["Imagem"], list) and len(item["fields"]["Imagem"]) > 0:
                # Extrai o primeiro URL do array de imagens
                item["ImagemURL"] = item["fields"]["Imagem"][0].get("url", None)

        return item
    except Exception as e:
        print(f"Erro ao processar item: {e}")
        return item

In [ ]:
newDf = dataframeBaseEventos.map(f=recInsights)

In [ ]:
print(newDf.count())  # Verifica se o número de registros não mudou

# Exibir alguns registros transformados
newDf.toDF().select("ImagemURL").show(10, truncate=False)

Esses 3 blocos abaixo são para tirar a url de dentro de Imagens

In [5]:
newDf = dataframeBaseEventos.toDF()  # Converter DynamicFrame para DataFrame

# Exibir o schema real da estrutura de dados
newDf.printSchema()

root
 |-- id: string (nullable = true)
 |-- createdTime: string (nullable = true)
 |-- fields: struct (nullable = true)
 |    |-- Título do evento: string (nullable = true)
 |    |-- Categoria de evento: string (nullable = true)
 |    |-- Data e Horário: string (nullable = true)
 |    |-- Tipo do evento: string (nullable = true)
 |    |-- Link de inscrição: string (nullable = true)
 |    |-- Formato: string (nullable = true)
 |    |-- Trimestre: string (nullable = true)
 |    |-- Semestre: string (nullable = true)
 |    |-- Publico: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Hub: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Imagem: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- width: integer (nullable = true)
 |    |    |    |-- height: integer (nullable = true)
 |    |    |    |-- url: string (nullable

In [6]:
newDf.select("fields.Imagem").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
from pyspark.sql.functions import col, explode

# Explodir a coluna "Imagem" para acessar o primeiro item do array
newDf = newDf.withColumn("Imagem", explode(col("fields.Imagem")))

# Agora, acessar a URL corretamente
newDf = newDf.withColumn("Imagem", col("Imagem.url"))

# Mostrar resultado
newDf.select("Imagem").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Imagem                                                                                                                                                                                                                                                                                                                                                                    |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Essa parte do bloco acima são testes

In [ ]:
newDf = dataframeBaseEventos.apply_mapping(mapping).resolveChoice([("Evento Privado?", "cast:boolean"), ("Imagem", "cast:string")]) \
        .drop_fields(["id", "Tipo do evento", "createdTime", "Hub", "Publico", "Trimestre", "Semestre", "Categoria de evento", "Status do evento", 
                      "Created By", "Nome do criador do evento", "Created"])

In [ ]:
newDf.printSchema()

In [ ]:
newDf.toDF().show(5)

In [ ]:
try:
    connection = psycopg2.connect(
        host="db-dev-poc-rds.c3dnragwy3og.us-east-1.rds.amazonaws.com",
        port="5432",
        database="db_dev_poc_rds",
        user="postgres",
        password="P06n9VDnXZ5CLlg9UNrgd6h53"
    )
    print("Conexão bem-sucedida!")
    connection.close()
except Exception as e:
    print(f"Erro ao conectar ao banco: {e}")

In [ ]:
jdbc_url = "jdbc:postgresql://db-dev-poc-rds.c3dnragwy3og.us-east-1.rds.amazonaws.com:5432/db_dev_poc_rds"
jdbc_properties = {
    "user": "postgres",
    "password": "P06n9VDnXZ5CLlg9UNrgd6h53",
    "driver": "org.postgresql.Driver",
    "stringtype": "unspecified"
}

In [ ]:
from pyspark.sql.functions import col, to_timestamp

newDf = newDf.toDF()  # Convertendo DynamicFrame para DataFrame

newDf = newDf.select(
    col("Título do evento").alias("name"),
    col("Link de inscrição").alias("url"),
    col("Formato").alias("modality"),
    to_timestamp(col("Data e Horário")).alias("start_date"),  # Conversão correta de data
    to_timestamp(col("Data e Horário de Término")).alias("end_date"),  # Conversão correta de data
    col("Imagem").alias("image"),
    col("Descrição").alias("detail"),
    col("Ativo").cast("boolean").alias("active"),
    col("Listar em Eventos Recomendados na Plataforma?").cast("boolean").alias("is_main_event"),
    col("Evento Privado?").cast("boolean").alias("private_event"), 
    col("Publicado?").cast("boolean").alias("published"), 
    col("Label").alias("id"),
    col("Local do Evento").alias("address"), 
    col("Evento Destaque(site)").cast("boolean").alias("featured_events"), 
    col("Tipo de Ingresso(site)").alias("ticket_type"),
    col("Local(Sala/Andar)(site)").alias("room_and_floor"),
    col("Tema(site)").alias("theme"),
    col("Resumo do tema(site)").alias("theme_summary"), 
    col("Programação(site)").alias("schedule"),
    col("Palestrantes(site)").alias("speakers")
)


In [ ]:
newDf.show(truncate=False)

In [ ]:
# from pyspark.sql.functions import col, to_timestamp

# newDf = newDf.toDF()

# newDf = newDf.withColumn("start_date", to_timestamp(col("Data e Horário"))) \
#              .withColumn("end_date", to_timestamp(col("Data e Horário de Término")))

# newDf.select("start_date", "end_date").show(truncate=False)

In [ ]:
# from pyspark.sql.functions import col, to_timestamp

# # Ajustando os tipos e renomeando as colunas
# newDf = newDf.toDF().select(
#     col("Título do evento").alias("name"),
#     col("Link de inscrição").alias("url"),
#     col("Formato").alias("modality"),
#     col("Imagem").alias("image"),
#     col("Descrição").alias("detail"),
#     col("Ativo").alias("active"),
#     col("Listar em Eventos Recomendados na Plataforma?").alias("is_main_event"),
#     col("Evento Privado?").alias("private_event"), 
#     col("Publicado?").alias("published"), 
#     col("Label").alias("id"),
#     col("Local do Evento").alias("address"), 
#     col("Evento Destaque(site)").alias("featured_events"), 
#     col("Tipo de Ingresso(site)").alias("ticket_type"),
#     col("Local(Sala/Andar)(site)").alias("room_and_floor"),
#     col("Tema(site)").alias("theme"),
#     col("Resumo do tema(site)").alias("theme_summary"), 
#     col("Programação(site)").alias("schedule"),
#     col("Palestrantes(site)").alias("speakers")
#     #col("Label").cast("boolean").alias("is_founder")
# )

In [ ]:
newDf.printSchema()

In [ ]:
newDf.select("start_date", "end_date").show(truncate=False)

In [ ]:
# newDf.limit(1).write.jdbc(
#     url=jdbc_url,
#     table="events_glue",
#     mode="append",
#     properties=jdbc_properties
# )

In [ ]:
newDf.write.jdbc(
        url=jdbc_url,
        table="events_glue",
        mode="append",  # Adiciona os registros sem sobrescrever
        properties=jdbc_properties
)